# Disclaimer: The tutorial below is a part of the Institute for Behavioral Genetics International Statistical Genetics 2021 Workshop. 

### If you would like additional content beyond this tutorial, please check out the following YouTube videos detailing Hail from the Broad team:
* [Sequencing and Hail](https://youtu.be/2N_VqmX22Xg)
* [BroadE: HAIL - Practical](https://www.youtube.com/watch?v=LlJZ4NGtIkY)

In this practical, you will learn how to:

1) Use simple python code and Jupyter notebooks.

2) Use Hail to import a VCF and run basic queries over sequencing data.

# Introduction


### It doesn't all need to "stick" today

This practical contains a lot of new material, and the goal of this workbook is not for you to be able to reproduce from memory all the various capabilities demonstrated here. Instead, the goal is for you to get a sense for the kind of analysis tasks that sequencing data requires, and to gain some exposure to what these analyses look like in Hail. 

There is no one-size-fits-all sequencing analysis pipeline, because each sequencing dataset will have unique properties that need to be understood and accounted for in QC and analysis. Hail can empower you to interrogate sequencing data, but it cannot give you all the questions to ask!

Some of the questions and exercises in this notebook might seem unrelated to the specific task of analyzing sequencing data, but that is intentional -- Hail is a computational tool that hopes to help you indulge your scientific curiosity, and asking and answering a variety of questions about many aspects of your data is the best way to learn *how to Hail*.

We don't expect you to be able to run a full GWAS on your own data in Hail tomorrow. If this is something you want to do, there are **lots more** resources available -- documentation, cookbooks, tutorials, and most importantly, the Hail community on the [forum](https://discuss.hail.is) and [zulip chatroom](https://hail.zulipchat.com).

### We encourage you to play

Hail is a highly expressive library with lots of functionality -- you'll see just a small fraction of it today. Throughout this notebook and especially in the denoted **exercises**, we encourage you to experiment with the code being run to see what happens! Sometimes it will be an error, but sometimes you will encounter new pieces of functionality. If you're curious about how to use Hail to ask slightly different questions than the code or exercises here, please ask the faculty! We are eager to help.

# 1. Using Jupyter

The notebook software that you are using right now is called [Jupyter](https://jupyter.org/), which came from a combination of the languages **Ju**lia, **Pyt**hon, and **R**.

**Learning objectives**

 - be comfortable running, editing, adding, and deleting code cells.
 - learn techniques for unblocking yourself if Jupyter acts up.

### Running cells
Evaluate cells using SHIFT + ENTER. Select the next cell and run it. If you prefer clicking, you can select the cell and click the "Run" button in the toolbar above.

In [1]:
print('Hello, world')

Hello, world


### Modes

Jupyter has two modes, a **navigation mode** and an **editor mode**.

#### Navigation mode:

 - <font color="blue"><strong>BLUE</strong></font> cell borders
 - `UP` / `DOWN` move between cells
 - `ENTER` while a cell is selected will move to **editing mode**.
 - Many letters are keyboard shortcuts! This is a common trap.
 
#### Editor mode:

 - <font color="green"><strong>GREEN</strong></font> cell borders
 - `UP` / `DOWN`/ move within cells before moving between cells.
 - `ESC` will return to **navigation mode**.
 - `SHIFT + ENTER` will evaluate a cell and return to **navigation mode**.
 
Try editing this markdown cell by double clicking, then re-rendering it by "running" the cell.

### Cell types

There are several types of cells in Jupyter notebooks. The two you will see in this notebook are **Markdown** (text) and **Code**.

In [2]:
# This is a code cell
my_variable = 5

**This is a markdown cell**, so even if something looks like code (as below), it won't get executed!

my_variable += 1

### Shell commands

It is possible to call command-line utilities from Jupyter by prefixing a line with a `!`. For instance, we can print the current directory:

In [3]:
! pwd

/home/jupyter-hpatel96/easyGWAS/notebooks/utils


### Tips and tricks

Keyboard shortcuts:

 - `SHIFT + ENTER` to evaluate a cell
 - `ESC` to return to navigation mode
 - `y` to turn a markdown cell into code
 - `m` to turn a code cell into markdown
 - `a` to add a new cell **above** the currently selected cell
 - `b` to add a new cell **below** the currently selected cell
 - `d, d` (repeated) to delete the currently selected cell
 - `TAB` to activate code completion
 
To try this out, create a new cell below this one using `b`, and print `my_variable` by starting with `print(my` and pressing `TAB`!

## <strong style="color: red;">Resetting Jupyter if you get stuck</strong>

If at any point during this practical, you are unable to successfully run cells, it is possible that your Python interpreter is in a bad state due to cells being run in an incorrect order. If this happens, you can recover a working session by doing the following:

1. Navigate to the "Kernel" menu at the top, and select "Restart and clear output".

2. Select the cell you were working on, then select "Run all above" from the "Cell" menu at the top.

3. If the problem persists, reach out to the faculty for help!

# 2. Import and initialize Hail

In addition to Hail, we import a few methods from the Hail plotting library. We'll see examples soon!

In [4]:
import hail as hl
from hail.plot import output_notebook, show

Now we initialize Hail and set up plotting to display inline in the notebook.

In [5]:
hl.init()
output_notebook()

2023-02-05 18:31:33.172 WARN  NativeCodeLoader:60 - Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
Running on Apache Spark version 3.1.3
SparkUI available at http://hub-testing.c.metal-bonus-375300.internal:4040
Welcome to
     __  __     <>__
    / /_/ /__  __/ /
   / __  / _ `/ / /
  /_/ /_/\_,_/_/_/   version 0.2.108-fc03e9d5dc08
LOGGING: writing to /home/jupyter-hpatel96/easyGWAS/notebooks/utils/hail-20230205-1831-0.2.108-fc03e9d5dc08.log


Loading BokehJS ...

This notebook works on a small (~16MB) downsampled chunk of the publically available Human Genome Diversity Project (HGDP) dataset. HGDP is a super-set of the well-known [1000 genomes](https://www.internationalgenome.org/) dataset, with a broader group of represented populations.

We can see the files used using `ls` below:

In [6]:
! ls -lh resources/

total 19M
-rw-r--r-- 1 jupyter-hpatel96 jupyter-hpatel96  16K Feb  5 18:30 HGDP_sample_data.tsv
-rw-r--r-- 1 jupyter-hpatel96 jupyter-hpatel96 2.6M Feb  5 18:30 ensembl_gene_annotations.txt
-rw-r--r-- 1 jupyter-hpatel96 jupyter-hpatel96 392K Feb  5 18:30 hgdp_gene_annotations.tsv
-rw-r--r-- 1 jupyter-hpatel96 jupyter-hpatel96  16M Feb  5 18:31 hgdp_subset.vcf.bgz


# 3. Explore genetic data with Hail

#### Learning Objectives:

- To be comfortable exploring Hail data structures, especially the `MatrixTable`.
- To understand categories of functionality for performing QC.

### Import data from VCF

The [Variant Call Format (VCF)](https://en.wikipedia.org/wiki/Variant_Call_Format) is a common file format for representing genetic data collected on multiple individuals (samples).

Hail has an [import_vcf](https://hail.is/docs/0.2/methods/impex.html#hail.methods.import_vcf) function that reads this file to a Hail `MatrixTable`, which is a general-purpose data structure that is often used to represent a matrix of genetic data.

Why not work directly on the VCF? While VCF is a text format that is easy for humans to read, it is inefficient to process on a computer. 

The first thing we do is import (`import_vcf`) and convert the `VCF` file into a Hail native file format. This is done by using the `write` method below. Any queries that follow will now run much more quickly.

In [7]:
hl.import_vcf('resources/hgdp_subset.vcf.bgz', min_partitions=4, reference_genome='GRCh38')\
.write('resources/hgdp.mt', overwrite=True)

2023-02-05 18:31:54.900 Hail: INFO: scanning VCF for sortedness...
2023-02-05 18:32:02.032 Hail: INFO: Coerced sorted VCF - no additional import work to do
2023-02-05 18:32:11.816 Hail: INFO: wrote matrix table with 10441 rows and 392 columns in 4 partitions to resources/hgdp.mt


### HGDP as a Hail `MatrixTable`

We represent genetic data as a Hail [`MatrixTable`](https://hail.is/docs/0.2/overview/matrix_table.html), and name our variable `mt` to indicate this.

In [8]:
mt = hl.read_matrix_table('resources/hgdp.mt')

### What is a `MatrixTable`?

Let's explore it!

You can see:
 - **numeric** types:
     - integers (`int32`, `int64`), e.g. `5`
     - floating point numbers (`float32`, `float64`), e.g. `5.5` or `3e-8`
 - **strings** (`str`), e.g. `"Foo"`
 - **boolean** values  (`bool`) e.g. `True`
 - **collections**:
     - arrays (`array`), e.g. `[1,1,2,3]`
     - sets (`set`), e.g. `{1,3}`
     - dictionaries (`dict`), e.g. `{'Foo': 5, 'Bar': 10}`
 - **genetic data types**:
     - loci (`locus`), e.g. `[GRCh37] 1:10000` or `[GRCh38] chr1:10024`
     - genotype calls (`call`), e.g. `0/2` or `1|0`

In [10]:
mt.describe(widget=True)

### Exercise
Take a few moments to explore the interactive representation of the matrix table above.

* Where is the variant information (`locus` and `alleles`)? 
<details><summary>Solution</summary>In the rows of the matrix.</details>
* Where is the sample identifier (`s`)?<details><summary>Solution</summary>In the columns of the matrix.</details>
* Where is the genotype quality `GQ`?<details><summary>Solution</summary>In the entries of the matrix.</details>

### `show`

Hail has a variety of functionality to help you quickly interrogate a dataset. The `show()` method prints the first few values of any field, and even prints in pretty HTML output in a Jupyter notebook! 

In [11]:
mt.s.show()

""
s
str
"""LP6005441-DNA_F08"""
"""LP6005441-DNA_C05"""
"""HGDP00961"""
"""HGDP00804"""
"""HGDP00926"""
"""HGDP00716"""
"""HGDP01269"""
"""HGDP00241"""


It is also possible to show() the matrix table itself, which prints a portion of the top-left corner of the variant-by-sample matrix:

In [12]:
# show() works fine with no arguments, but can print too little data by default on small screens!
mt.show(n_cols=3)

+---------------+------------+------------------------+------------------------+
| locus         | alleles    | 'LP6005441-DNA_F08'.GT | 'LP6005441-DNA_F08'.DP |
+---------------+------------+------------------------+------------------------+
| locus<GRCh38> | array<str> | call                   |                  int32 |
+---------------+------------+------------------------+------------------------+
| chr1:17379    | ["G","A"]  | 0/0                    |                     11 |
| chr1:95068    | ["G","A"]  | 0/0                    |                     15 |
| chr1:111735   | ["C","A"]  | 0/0                    |                     14 |
| chr1:134610   | ["G","A"]  | 0/0                    |                      8 |
| chr1:414783   | ["T","C"]  | NA                     |                     NA |
| chr1:1130877  | ["C","G"]  | 0/0                    |                     24 |
| chr1:1226707  | ["C","G"]  | 0/0                    |                     26 |
| chr1:1491494  | ["G","A"]  | 0/0                    |                     27 |
| chr1:1618118  | ["G","A"]  | 0/0                    |                     27 |
| chr1:2078529  | ["G","A"]  | 0/0                    |                     30 |
+---------------+------------+------------------------+------------------------+

+------------------------+------------------------+------------------------+
| 'LP6005441-DNA_F08'.GQ | 'LP6005441-DNA_F08'.AD | 'LP6005441-DNA_F08'.PL |
+------------------------+------------------------+------------------------+
|                  int32 | array<int32>           | array<int32>           |
+------------------------+------------------------+------------------------+
|                     20 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     NA | NA                     | NA                     |
|                     40 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     40 | NA                     | NA                     |
|                     20 | NA                     | NA                     |
|                     50 | NA                     | NA                     |
+------------------------+------------------------+------------------------+

+------------------------+------------------------+------------------------+
| 'LP6005441-DNA_C05'.GT | 'LP6005441-DNA_C05'.DP | 'LP6005441-DNA_C05'.GQ |
+------------------------+------------------------+------------------------+
| call                   |                  int32 |                  int32 |
+------------------------+------------------------+------------------------+
| 0/0                    |                     15 |                     30 |
| 0/0                    |                     27 |                     30 |
| 0/0                    |                     15 |                     20 |
| NA                     |                     NA |                     NA |
| 0/0                    |                      9 |                     20 |
| 0/0                    |                     27 |                     30 |
| 0/0                    |                     25 |                     20 |
| 0/0                    |                     30 |                     20 |
| 0/0                    |                     30 |                     40 |
| 0/0                    |                     37 |                     60 |
+------------------------+------------------------+------------------------+

+------------------------+------------------------+----------------+
| 'LP6005441-DNA_C05'.AD | 'LP6005441-DNA_C05'.PL | 'HGDP00961'.GT |
+------------------------+------------------------+----------------+
| array<int32>           | arr

The above output is visually noisy because the matrix table has as lot of information in it. `show`ing just the called genotype (`GT`) is a bit more friendly.

The printed representation of GT is explained below, where `a` is the reference allele and `A` is the alternate allele:

`0/0` : homozygous reference or `aa`

`0/1` : heterozygous or `Aa`

`1/1` : homozygous alternate or `AA` 


In [13]:
mt.GT.show()

,,,,,
,,'LP6005441-DNA_F08','LP6005441-DNA_C05','HGDP00961','HGDP00804'
locus,alleles,GT,GT,GT,GT
locus<GRCh38>,array<str>,call,call,call,call
chr1:17379,"[""G"",""A""]",0/0,0/0,0/0,0/0
chr1:95068,"[""G"",""A""]",0/0,0/0,NA,0/0
chr1:111735,"[""C"",""A""]",0/0,0/0,0/1,0/0
chr1:134610,"[""G"",""A""]",0/0,NA,NA,0/0
chr1:414783,"[""T"",""C""]",NA,0/0,NA,NA
chr1:1130877,"[""C"",""G""]",0/0,0/0,0/0,0/0
chr1:1226707,"[""C"",""G""]",0/0,0/0,0/0,0/0


### Exercise

There is a fourth value seen above, other than `0/0`, `0/1`, `1/1`. What is it?

<details><summary>Solution</summary>You also see `NA` values. These are missing values, where there is not enough data to make a genotype call for a given sample.</details>

### `summarize`
`summarize` Prints (potentially) useful information about any field or object:

`DP` is the read depth (number of short reads spanning a position for a given sample). Let's summarize all values of DP:

In [14]:
mt.DP.summarize()

Non-missing,3851673 (94.11%)
Missing,241199 (5.89%)
Minimum,0
Maximum,5057
Mean,33.02
Std Dev,30.20


`AD` is the array of allelic depth per allele at a called genotype. Note especially the missingness properties:

In [15]:
mt.AD.summarize()

Non-missing,1164892 (28.46%)
Missing,2927980 (71.54%)
Min Size,2
Max Size,2
Mean Size,2.00
Non-missing,2329784 (100.00%)
Missing,0
Minimum,0
Maximum,1299
Mean,17.09
Std Dev,15.13


### Exercise

In the empty cell below, summarize some of the other fields on the matrix table. You can use the interactive widget above to find the names of some of the other fields.

Share any interesting findings with your colleagues!

In [16]:
# a few solutions
mt.locus.summarize()
mt.info.QD.summarize()

Non-missing,10441 (100.00%)
Missing,0
Contig Counts,"{'chr1': 881, 'chr10': 516, 'chr11': 483, 'chr12': 411, 'chr13': 397, 'chr14': 332, 'chr15': 316, 'chr16': 319, 'chr17': 312, 'chr18': 270, 'chr19': 252, 'chr2': 799, 'chr20': 263, 'chr21': 194, 'chr22': 170, 'chr3': 728, 'chr4': 659, 'chr5': 618, 'chr6': 572, 'chr7': 576, 'chr8': 525, 'chr9': 476, 'chrX': 361, 'chrY': 11}"


Non-missing,10407 (99.67%)
Missing,34 (0.33%)
Minimum,0.56
Maximum,62.25
Mean,16.07
Std Dev,6.28


### `count`

`MatrixTable.count` returns a tuple with the number of rows (variants) and number of columns (samples).

In [17]:
mt.count()

(10441, 392)

The count above tells us that we have 10,441 variants and 392 samples. This is just a tiny slice of a real sequencing dataset. The largest sequencing datasets today comprise hundreds of thousands of samples and more than a billion variants.

## Hail has a large library of genetics functionality

Hail can be used to analyze any kind of data (Hail team members have used Hail to analyze household financial data, USA election polling data, and even to build a bot that posts real-time updates about the Euro 2020 tournament to Slack). However, Hail does not have *only* general-purpose analysis functionality. Hail has a large set of functionality built for genetics and genomics.

For example, `hl.summarize_variants` prints useful statistics about the variants in the dataset. These are not part of the generic `summarize()` function, which must support all kinds of data, not just variant data!

In [18]:
hl.summarize_variants(mt)

Number of alleles,Count
2,10441
Allele type,Count
SNP,10441
Metric,Value
Transitions,6602
Transversions,3839
Ratio,1.72
Contig,Count
chr1,881
chr2,799


# 4. Annotation

## Integrate sample information

Our dataset currently only has sample IDs and genetic data. In order to run a toy GWAS, we need phenotype information.

We can find it in the following file:

In [19]:
! head resources/HGDP_sample_data.tsv

sample_id	pop	continental_pop	sex_karyotype	sleep_duration	tea_intake_daily	general_happiness	screen_time_per_day
HG00107	gbr	nfe	XY	6	3	3.2895e+00	11
HG00114	gbr	nfe	XY	5	3	3.5099e+00	10
HG00121	gbr	nfe	XX	6	6	2.0851e+00	6
HG00127	gbr	nfe	XX	6	3	2.7580e+00	6
HG00132	gbr	nfe	XX	5	6	2.2454e+00	5
HG00149	gbr	nfe	XY	5	6	2.8159e+00	9
HG00177	fin	fin	XX	7	9	3.3661e+00	8
HG00190	fin	fin	XY	5	6	2.9159e+00	6
HG00233	gbr	nfe	XX	8	3	3.9002e+00	10


We can import it as a [Hail Table](https://hail.is/docs/0.2/overview/table.html) with [hl.import_table](https://hail.is/docs/0.2/methods/impex.html?highlight=import_table#hail.methods.import_table).

We call it `sd` for "sample data".

In [20]:
sd = hl.import_table('resources/HGDP_sample_data.tsv',
                     key='sample_id',
                     impute=True)

2023-02-05 18:39:45.569 Hail: INFO: wrote table with 393 rows in 1 partition to /tmp/persist_table8kYP1sbtGC
2023-02-05 18:39:46.184 Hail: INFO: Reading table to impute column types
2023-02-05 18:39:47.241 Hail: INFO: Finished type imputation
  Loading field 'sample_id' as type str (imputed)
  Loading field 'pop' as type str (imputed)
  Loading field 'continental_pop' as type str (imputed)
  Loading field 'sex_karyotype' as type str (imputed)
  Loading field 'sleep_duration' as type int32 (imputed)
  Loading field 'tea_intake_daily' as type int32 (imputed)
  Loading field 'general_happiness' as type float64 (imputed)
  Loading field 'screen_time_per_day' as type int32 (imputed)


The "key" argument tells Hail to use the `sample_id` field as the table key, which is used to find matching values in  joins. In a moment, we will be joining the `sd` table onto our matrix table so that we can use the sample data fields in our QC and analysis. It is also possible to specify a new key for an existing table using the `.key_by(...)` method.

The "impute" argument tells Hail to impute the data types of the fields on the table. What does this mean? It means that you can ask Hail to figure out what is the data type in each column field such as `str` (string or just characters), `bool` (boolean or just true and false), `float64` (float or numbers with decimals), or `int32` (integer or numbers without decimals/whole numbers). If you don't use the `impute` flag or specify types manually with the `types` argument, each field will be imported as a string.

While we can see the names and types of fields in the logging messages and in the `head` output above, we can also `show` this table:

In [21]:
sd.show()

,,,,,,,
sample_id,pop,continental_pop,sex_karyotype,sleep_duration,tea_intake_daily,general_happiness,screen_time_per_day
str,str,str,str,int32,int32,float64,int32
"""HG00107""","""gbr""","""nfe""","""XY""",6,3,3.29e+00,11
"""HG00114""","""gbr""","""nfe""","""XY""",5,3,3.51e+00,10
"""HG00121""","""gbr""","""nfe""","""XX""",6,6,2.09e+00,6
"""HG00127""","""gbr""","""nfe""","""XX""",6,3,2.76e+00,6
"""HG00132""","""gbr""","""nfe""","""XX""",5,6,2.25e+00,5
"""HG00149""","""gbr""","""nfe""","""XY""",5,6,2.82e+00,9
"""HG00177""","""fin""","""fin""","""XX""",7,9,3.37e+00,8
"""HG00190""","""fin""","""fin""","""XY""",5,6,2.92e+00,6


And we can `summarize` each field in `sd`:

In [22]:
sd.summarize()

2023-02-05 18:39:49.982 Hail: INFO: Coerced sorted dataset


Non-missing,392 (100.00%)
Missing,0
Min Size,7
Max Size,17
Mean Size,7.92
Sample Values,"['HG00107', 'HG00114', 'HG00121', 'HG00127', 'HG00132']"
Non-missing,392 (100.00%)
Missing,0
Min Size,2
Max Size,11
Mean Size,3.84


## Add sample data to our HGDP `MatrixTable`

Let's now merge our genetic data (`mt`) with our sample data (`sd`).

This is a join between the `sd` table and the columns of our matrix table. It just takes one line:

In [23]:
mt = mt.annotate_cols(sample_data = sd[mt.s])

### What's going on here?

Understanding what's going on here is a bit more difficult. To understand, we need to understand a few pieces:

#### 1. `annotate` methods

In Hail, `annotate` methods refer to **adding new fields**. 

 - `MatrixTable`'s `annotate_cols` adds new column (**sample**) fields.
 - `MatrixTable`'s `annotate_rows` adds new row (**variant**) fields.
 - `MatrixTable`'s `annotate_entries` adds new entry (**genotype**) fields.
 - `Table`'s `annotate` adds new row fields.

In the above cell, we are adding a new column (**sample**) field called "sample_data". This field should be the values in our table `sd` associated with the sample ID `s` in our `MatrixTable` - that is, this is performing a **join**.

Python uses square brackets to look up values in dictionaries:

    >>> d = {'foo': 5, 'bar': 10}
    
    >>> d['foo']
    'bar'

You should think of this in much the same way - for each column of `mt`, we are looking up the fields in `sd` using the sample ID `s`.

Let's see how the matrix table has changed:

In [24]:
mt.describe(widget=True)

### Cheat sheets

More information about matrix tables and tables can be found in a graphical representation as Hail cheat sheets:

 - [MatrixTable](https://hail.is/docs/0.2/_static/cheatsheets/hail_matrix_tables_cheat_sheet.pdf)
 - [Table](https://hail.is/docs/0.2/_static/cheatsheets/hail_tables_cheat_sheet.pdf)

## Query the sample data

We will use some of the general-purpose query functionality to interrogate the sample data we have imported.

The code below uses the `aggregate_cols` method on our matrix table, which computes aggregate statistics about column (sample) data. There are also methods for `aggregate_rows` (aggregate over row data) and `aggregate_entries` aggregate over all of the entries in our matrix, one per variant per sample).

Hail **aggregators** can be recognized by the `hl.agg` prefix. Some examples:

 - `hl.agg.fraction(CONDITION)` - compute the fraction of values at which `CONDITION` is true.
 - `hl.agg.count_where(CONDITION)` - compute the number of values at which `CONDITION` is true.
 - `hl.agg.stats(X)` - compute a few useful statistics about `X`.
 - `hl.agg.counter(X)` - compute the number of occurrences of each unique value of `X`. Useful for categorical fields like strings, not as useful for numbers!
 - `hl.agg.corr(X, Y)` - compute the Pearson correlation coefficient between X and Y values.
 - For more adventurous students, see the [full list of aggregators](https://hail.is/docs/0.2/aggregators.html#sec-aggregators).


### Sex karyotype

To start, we will compute the occurrences of each value of `sex_karyotype`:

In [25]:
mt.aggregate_cols(hl.agg.counter(mt.sample_data.sex_karyotype))

{'XX': 188, 'XY': 204}

The above result tells us that slightly more than half of our samples are XY, and the rest are XX. What should you do if some of your samples are neither XX or XY? That depends on the analysis you are trying to do, but you should be ready to think about this case!

### Ancestry

How many people are in each self-reported major continental ancestry group?

In [26]:
mt.aggregate_cols(hl.agg.counter(mt.sample_data.continental_pop))

{'afr': 93,
 'amr': 48,
 'eas': 72,
 'fin': 8,
 'mid': 16,
 'nfe': 60,
 'oth': 8,
 'sas': 87}

### Exercise

Try changing `continental_pop` to `pop` and rerunning the cell above. Most of the populations are abbreviated, but see if you can find an ancestral population from each continent among the non-abbreviated ones!

### Numeric aggregations

The numeric aggregators are used the same way:

In [27]:
mt.aggregate_cols(hl.agg.stats(mt.sample_data.sleep_duration))

Struct(mean=6.030612244897959, stdev=1.44246162636083, min=2.0, max=10.0, n=392, sum=2364.0)

### Exercise

Use the `fraction` and `count_where` aggregators to answer the following questions in the cells below:

1. How many samples drink more than 8 cups of tea per day? *Hint: the CONDITION will take the form `SOMETHING > 8`.*

2. What fraction of samples sleep less than 4 hours per day?


In [28]:
# solution to Q1
mt.aggregate_cols(hl.agg.count_where(mt.sample_data.tea_intake_daily > 8))

46

In [29]:
# solution to Q2
mt.aggregate_cols(hl.agg.fraction(mt.sample_data.sleep_duration < 4))

0.03826530612244898

## The end! 

If you still have time and desire, feel free to explore the YouTube links at the top of this tutorial or from their [Github](https://github.com/mkveerapen/2021_IBG_Hail). From there, you can access more practical content from the Broad and Hail team! Additionally, you can find tutorial content directly on the [Hail webpage](https://hail.is/docs/0.2/tutorials-landing.html).